In [41]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import os

In [42]:
EVAL_FOLDER = '../eval'
EVAL_FILE = '../eval/eval_model_kf4_9fb9ac_fc.csv'

In [43]:
eval_file_list = [f for f in os.listdir(EVAL_FOLDER) if f.startswith("eval")]
print (len(eval_file_list))
for f in eval_file_list:
    print(f)

24
eval_model_kf4_9fb9ac_fc.csv
eval_model_kf4_b58461_b2.csv
eval_model_kf4_d6d1d9_34.csv
eval_model_kf4_253ddf_66.csv
eval_model_kf4_145404_85.csv
eval_model_kf4_059637_87.csv
eval_model_kf4_fcb630_ee.csv
eval_model_kf4_059637_09.csv
eval_model_kf4_059637_d7.csv
eval_model_kf4_253ddf_14.csv
eval_model_kf4_d6d1d9_bf.csv
eval_model_kf4_6f1a27_da.csv
eval_model_kf4_9fb9ac_cd.csv
eval_model_kf4_b1a73d_23.csv
eval_model_kf4_63fd5a_e4.csv
eval_model_kf4_059637_0c.csv
eval_model_kf4_b1a73d_d0.csv
eval_model_kf4_fcb630_0b.csv
eval_model_kf4_fdb9c2_52.csv
eval_model_kf4_6f1a27_73.csv
eval_model_kf4_145404_c8.csv
eval_model_kf4_b58461_c8.csv
eval_model_kf4_fdb9c2_83.csv
eval_model_kf4_63fd5a_ba.csv


In [44]:
df_eval = pd.read_csv(EVAL_FILE)
df_eval.head()

,Id,Ground_Truth,Predictions
0,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18,18
1,003957a8-bbb7-11e8-b2ba-ac1f6b6435d0,25,0 16 25
2,003feb6e-bbca-11e8-b2bc-ac1f6b6435d0,0,0
3,004a2b84-bbc4-11e8-b2bc-ac1f6b6435d0,0 25,0 25
4,004f7c66-bb9b-11e8-b2b9-ac1f6b6435d0,25,25


In [47]:
df_gt = df_eval[['Id','Ground_Truth']].copy()
df_gt.head()

,Id,Ground_Truth
0,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18
1,003957a8-bbb7-11e8-b2ba-ac1f6b6435d0,25
2,003feb6e-bbca-11e8-b2bc-ac1f6b6435d0,0
3,004a2b84-bbc4-11e8-b2bc-ac1f6b6435d0,0 25
4,004f7c66-bb9b-11e8-b2b9-ac1f6b6435d0,25


In [48]:
df_pred = df_eval[['Id','Predictions']].copy()
df_pred.head()

,Id,Predictions
0,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18
1,003957a8-bbb7-11e8-b2ba-ac1f6b6435d0,0 16 25
2,003feb6e-bbca-11e8-b2bc-ac1f6b6435d0,0
3,004a2b84-bbc4-11e8-b2bc-ac1f6b6435d0,0 25
4,004f7c66-bb9b-11e8-b2b9-ac1f6b6435d0,25


In [49]:
def make_label_array (labels):

    # set up a numpy array to receive the encoded label
    label_array = np.zeros(28)

    if isinstance(labels, str):
    
        # split the space separated multi-label into a list of individual labels
        split_label = labels.split(' ')

        # turn on the positive columns in the labels array
        for label in split_label:
            label_array[np.uint8(label)] = 1
    
    return label_array
    
# unit test 
la = make_label_array(df_gt['Ground_Truth'][52])
p = make_label_array(df_pred['Predictions'][52])
print(la.shape)
print(la)
print(p.shape)
print(p)


(28,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 0.]
(28,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0.]


In [55]:
def print_statistics(df_eval, df_gt, df_pred):
    
    tp = np.zeros(28)
    fp = np.zeros(28)
    tn = np.zeros(28)
    fn = np.zeros(28)
    count = np.zeros(28) 
    for i in range(len(df_eval)):
        gt = make_label_array(df_gt['Ground_Truth'][i])
        pred = make_label_array(df_pred['Predictions'][i])
        tp += gt * pred
        tn += (1 - gt) * (1 - pred)
        fp += (1 - gt) * pred
        fn += gt * (1 - pred)
        count += gt

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    '''
    print(' lbl   prec     recall   f1')
    for i in range(28):
        if f1[i] < 0.7:
            print('[{:2}]   {:.3f}    {:.3f}    {:.3f}'.format(i, precision[i], recall[i], f1[i]))
    '''
    return precision, recall, f1, count
            
# unit test
print_statistics(df_eval, df_gt, df_pred)

(array([7.98398836e-01, 7.15789474e-01, 6.35616438e-01, 4.74820144e-01,
        5.67441860e-01, 6.03550296e-01, 3.95348837e-01, 5.83061889e-01,
        5.00000000e-01, 3.33333333e-01, 2.00000000e-01, 5.67010309e-01,
        2.81553398e-01, 3.13953488e-01, 8.42592593e-01, 1.08108108e-02,
        1.01449275e-01, 6.97674419e-02, 4.00000000e-01, 4.45454545e-01,
        3.33333333e-01, 4.27574171e-01, 3.94736842e-01, 6.20481928e-01,
        6.25000000e-01, 5.68115942e-01, 1.89189189e-01, 6.31313131e-04]),
 array([0.84710425, 0.56666667, 0.63736264, 0.40993789, 0.67777778,
        0.40316206, 0.45535714, 0.62587413, 0.25      , 0.33333333,
        0.5       , 0.47413793, 0.453125  , 0.47368421, 0.82727273,
        1.        , 0.125     , 0.31578947, 0.34042553, 0.32236842,
        0.28571429, 0.70200573, 0.36585366, 0.65814696, 0.3030303 ,
        0.71532847, 0.2       , 1.        ]),
 array([0.82203072, 0.63255814, 0.63648834, 0.44      , 0.61772152,
        0.48341232, 0.42323651, 0.603709

In [51]:
print(EVAL_FOLDER + '/' + eval_file_list[0])
df_eval = pd.read_csv(EVAL_FOLDER + '/' + EVAL_FILE)
tmp = EVAL_FOLDER + '/' + eval_file_list[0]
tmp = tmp.split('_',2)[2][:16]
print(tmp)

../eval/eval_model_kf4_9fb9ac_fc.csv
kf4_9fb9ac_fc.cs


In [54]:
statistics = []

for eval_file in tqdm(eval_file_list):
    
    # load data
    df_eval = pd.read_csv(EVAL_FOLDER + '/' + eval_file)
    
    # create ground_truth data
    df_gt = df_eval[['Id','Ground_Truth']].copy()
    
    # create prediction data
    df_pred = df_eval[['Id','Predictions']].copy()
    
    #print('-----  Model Results for: {} -----'.format(eval_file))
    precision, recall, f1, _ = print_statistics(df_eval, df_gt, df_pred)
    #print(precision.shape)
    #print(recall.shape)
    #print(f1.shape)
    
    statistics.append([[eval_file], precision, recall, f1])

df_statistics = pd.DataFrame(np.array(statistics))
df_statistics.to_csv('../eval/statistics.csv', index=False, header=['model','precision','recall','f1'])
df_statistics.head()
    

100%|██████████| 24/24 [00:06<00:00,  3.60it/s]


,0,1,2,3
0,[eval_model_kf4_9fb9ac_fc.csv],"[0.7932659932659932, 0.8245614035087719, 0.788...","[0.9096525096525097, 0.7833333333333333, 0.769...","[0.8474820143884892, 0.8034188034188033, 0.778..."
1,[eval_model_kf4_b58461_b2.csv],"[0.7935528120713305, 0.8205128205128205, 0.801...","[0.8934362934362934, 0.8, 0.7335164835164835, ...","[0.8405375953505265, 0.810126582278481, 0.7661..."
2,[eval_model_kf4_d6d1d9_34.csv],"[0.8253400143163923, 0.9238095238095239, 0.828...","[0.8903474903474904, 0.8083333333333333, 0.782...","[0.8566121842496285, 0.8622222222222223, 0.805..."
3,[eval_model_kf4_253ddf_66.csv],"[0.8124076809453471, 0.8431372549019608, 0.733...","[0.8494208494208494, 0.7166666666666667, 0.711...","[0.8305020762551906, 0.7747747747747747, 0.722..."
4,[eval_model_kf4_145404_85.csv],"[0.8069686411149826, 0.8392857142857143, 0.844...","[0.8942084942084942, 0.7833333333333333, 0.763...","[0.8483516483516483, 0.8103448275862069, 0.802..."
